In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## क्लाइंट बनाना

इस उदाहरण में, हम [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) का उपयोग LLM तक पहुंचने के लिए करेंगे।

`model` को `gpt-4o-mini` के रूप में परिभाषित किया गया है। GitHub Models मार्केटप्लेस में उपलब्ध किसी अन्य मॉडल पर स्विच करके विभिन्न परिणाम देखने का प्रयास करें।

एक त्वरित परीक्षण के रूप में, हम एक साधारण प्रॉम्प्ट चलाएंगे - `What is the capital of France`।


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## फ़ंक्शन्स को परिभाषित करना

इस उदाहरण में, हम एजेंट को एक टूल तक पहुंच देंगे, जो एक फ़ंक्शन है जिसमें उपलब्ध छुट्टी स्थलों और उनकी उपलब्धता की सूची है।

आप इसे इस तरह सोच सकते हैं कि यह एक ऐसा परिदृश्य होगा जहां एक ट्रैवल एजेंट के पास उदाहरण के लिए, एक ट्रैवल डेटाबेस तक पहुंच हो सकती है।

जैसे-जैसे आप इस नमूने को पढ़ते हैं, आप नए फ़ंक्शन्स और टूल्स को परिभाषित करने की कोशिश कर सकते हैं, जिन्हें एजेंट कॉल कर सके।


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## फ़ंक्शन टूल को परिभाषित करना  
एजेंट को `vacation_destinations` को एक `FunctionTool` के रूप में उपयोग करने के लिए, हमें इसे एक के रूप में परिभाषित करना होगा।  

हम इस टूल का विवरण भी प्रदान करेंगे, जो एजेंट को यह पहचानने में मदद करता है कि उपयोगकर्ता द्वारा अनुरोधित कार्य के संबंध में उस टूल का उपयोग किस लिए किया जाता है।  


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## एजेंट को परिभाषित करना

अब हम नीचे दिए गए कोड में एजेंट बना सकते हैं। हम `system_message` को परिभाषित करते हैं ताकि एजेंट को यह निर्देश दिया जा सके कि वह उपयोगकर्ताओं को छुट्टी के गंतव्य खोजने में कैसे मदद करे। 

हम `reflect_on_tool_use` पैरामीटर को भी true पर सेट करते हैं। यह LLM को टूल कॉल के उत्तर को लेने और उसे प्राकृतिक भाषा में भेजने की अनुमति देता है। 

आप इस पैरामीटर को false पर सेट करके अंतर देख सकते हैं।


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## एजेंट चलाना

अब हम एजेंट को प्रारंभिक उपयोगकर्ता संदेश के साथ चला सकते हैं, जिसमें टोक्यो की यात्रा करने का अनुरोध किया गया है।

आप इस शहर के गंतव्य को बदल सकते हैं ताकि यह देखा जा सके कि एजेंट शहर की उपलब्धता पर कैसे प्रतिक्रिया देता है।


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
